In [ ]:
import sys
sys.path.append("/home/dictus/dmt/v2")
from neuro_dmt.library.users.hugo.validations import CellDensityValidation, INHRatioValidation

In [ ]:
import pandas as pd
example_cell_density_data = pd.DataFrame(
        {'layer': ['L1','L2','L3','L4','L5','L6'],
         "samples": [[100000, 110000, 90000],
                     [120000, 130000, 140000],
                     [110000, 200000, 60000],
                     [130000, 130000, 130000],
                     [100000, 130000, 160000],
                     [231000, 214000, 121000]]})


In [ ]:
from neuro_dmt.library.users.hugo.adapters import CircuitAdapter
circuit_config = "/gpfs/bbp.cscs.ch/project/proj68/circuits/O1/20190307/CircuitConfig"
circuit_model = CircuitAdapter(circuit_config)

In [ ]:
CellDensityValidation(data=example_cell_density_data)(circuit_model)

In [ ]:
INHRatioValidation(data=pd.DataFrame([{'layer': l} for l in['L1', 'L2', 'L3', 'L4', 'L5', 'L6']]))\
                        (circuit_model)

In [ ]:
# TODO: SynapseDensityValidation

In [ ]:
# automatically puts mtype in the title, rather than cluttering x-axis with repeated mentions
CellDensityValidation(data=pd.DataFrame([{'mtype': 'MC', 'layer': 'L{}'.format(l)} for l in range(1, 7)]))(circuit_model)

In [ ]:
class CellDensityByMtypeValidation(CellDensityValidation):
    
    class AdapterInterface(CellDensityValidation.AdapterInterface):
        
        def mtypes(adapter_self):
            """
            get all the mtypes represented in the circuit
            They are expected as strings without laye prefixes
            """
            pass

    def measurements_parameters(self, model):
        return [{'mtype': mtype} for mtype in model.mtypes()]

CellDensityByMtypeValidation()(circuit_model)
        

In [ ]:
circuit_model_2 = CircuitAdapter("/gpfs/bbp.cscs.ch/project/proj66/circuits/O1/r0/CircuitConfig")
CellDensityValidation(data=example_cell_density_data)(circuit_model, circuit_model_2)

In [ ]:
try:
    INHRatioValidation()
except AssertionError as E:
    print(E)

In [ ]:
from neuro_dmt.library.users.hugo.adapters.atlas import AtlasAdapter

In [ ]:
import yaml
DefilipeDatapath = "/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/DeFelipe2017.yaml"
with open(DefilipeDatapath) as DFDF:
    DFDict = yaml.load(DFDF)
DFDict

In [ ]:
import pandas as pd
DefilipeCellDensityData = pd.DataFrame({'layer':
                                        ["L1", "L2", "L3", "L4", "L5", "L6"],
                                       'samples': 
                                        [[subj['densities'][l] 
                                          for subj in DFDict['circuits'].values()]
                                         for l in range(6)]})
DefilipeCellDensityData

In [ ]:
rats_df =\
   pd.concat([
       pd.DataFrame(subject_data)\
         .assign(subject=subject_label)
       for subject_label, subject_data in DFDict['circuits'].items()])\
       .assign(layer=['L1', "L2", "L3", "L4", "L5", "L6"]*6).set_index("subject")\
       .rename(columns={"densities": "samples", "thicknesses": "thickness"})
print(rats_df.loc['P14-12'])
mean_rat = rats_df.groupby("layer").agg('mean')
print(mean_rat)
mean_rat['subject'] = "mean"
rats_df = pd.concat((rats_df, mean_rat.assign(subject="mean", layer=mean_rat.index).set_index("subject")))
print(rats_df)

In [ ]:
rat = "mean"
report = CellDensityValidation(data=rats_df.loc[rat].reset_index())(
    AtlasAdapter("/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/atlas/O1/DAN/"+rat))

In [ ]:
# test that CellDensityValidation's interface raises the right kind of error
class Amethodlessclass:
    pass
CellDensityValidation(data=example_cell_density_data)(Amethodlessclass())
# it doesn't work

In [ ]:
thal = AtlasAdapter("/gpfs/bbp.cscs.ch/project/proj55/entities/atlas/20190628/")
thal.label="Thalamus_atlas"
CellDensityValidation(data=pd.DataFrame({
    'samples' : [72802.84, 65364.87, 56678.06, 75391.47],
    'region': ['RT', '@VPL|VPM', 'VPL', 'VPM']
}))(thal)